In [21]:
import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from finta import TA
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import jaccard_score
import seaborn as sn
from tabulate import tabulate
from xgboost import XGBClassifier
from ta import add_all_ta_features
from sklearn.feature_selection import RFE

In [22]:
WINDOW = 8 # number of rows to look ahead to see what the price did
FETCH_INTERVAL = "60m"  # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
INTERVAL = '1y'     # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
symbol = 'AAPL'      # Symbol of the desired stock
ROWS_TO_PREDICT = 34
# one day 16 rows of data

In [23]:
data = pd.read_csv('C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\preprocess_new\\AAPL_2y_8_diff_11_05_2021 09_27_21_full.csv', index_col = 0)
data






,open,high,low,close,Adj Close,volume,open_1,open_2,open_3,open_4,...,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,close_shift,class_column
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-05-13 18:00:00-04:00,46.2350,46.4300,46.160000,46.237500,46.237500,0.0,-0.140000,-0.195000,-0.137501,-0.532502,...,46.237500,0.000000,0.000000,0.000000,0.000000,-48.174326,0.000000,0.000000,46.779999,1
2019-05-13 19:00:00-04:00,46.2425,46.2625,46.057500,46.057500,46.057500,0.0,0.007500,-0.132500,-0.187500,-0.130001,...,46.153711,0.000000,0.000000,0.000000,0.000000,-0.389294,-0.390054,-0.389294,46.930000,1
2019-05-14 04:00:00-04:00,46.5650,46.7350,46.565000,46.640000,46.640000,0.0,0.322500,0.330000,0.190000,0.135000,...,46.373640,0.000000,0.000000,0.000000,0.000000,1.264723,1.256793,0.870506,47.058250,0
2019-05-14 05:00:00-04:00,46.6250,46.7500,46.577500,46.750000,46.750000,0.0,0.060000,0.382500,0.390000,0.250000,...,46.539028,0.000000,0.000000,0.000000,0.000000,0.235849,0.235571,1.108408,47.320000,1
2019-05-14 06:00:00-04:00,46.7250,46.8250,46.725000,46.800000,46.800000,0.0,0.100000,0.160000,0.482500,0.490000,...,46.653430,0.000000,0.000000,0.000000,0.000000,0.106952,0.106895,1.216545,47.362499,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-10 15:30:00-04:00,127.1950,127.5000,126.809998,126.860001,126.860001,11140598.0,0.003502,-1.064995,-1.206795,-1.120003,...,127.577504,-2.392859,28.297455,11.574367,16.723087,-0.259456,-0.259793,174.366046,NaN,0
2021-05-10 16:00:00-04:00,126.8600,127.9881,117.824670,126.490000,126.490000,0.0,-0.335000,-0.331498,-1.399995,-1.541794,...,127.137846,-2.550077,17.244135,12.708321,4.535815,-0.291661,-0.292087,173.565829,NaN,0
2021-05-10 17:00:00-04:00,126.5000,126.8500,118.090096,126.430000,126.430000,0.0,-0.360000,-0.695000,-0.691498,-1.759995,...,126.852684,-2.783545,7.143102,11.595277,-4.452175,-0.047435,-0.047446,173.436064,NaN,0


In [24]:

# Class divide
data['class_column'].value_counts()

-1    2759
 1    2747
 0    2675
Name: class_column, dtype: int64

In [25]:
# del (data['close'])
del (data['close_shift'])
data = data.dropna()
train_set = data.iloc[:-ROWS_TO_PREDICT]
train_set = train_set.iloc[:-WINDOW] # optional drop last n rows (avoid of data leak)
test_set =data.iloc[-ROWS_TO_PREDICT:]

In [26]:
data['class_column'].value_counts()

-1    2759
 1    2746
 0    2675
Name: class_column, dtype: int64

In [27]:
train_set

,open,high,low,close,Adj Close,volume,open_1,open_2,open_3,open_4,...,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr,class_column
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-05-13 19:00:00-04:00,46.242500,46.262500,46.057500,46.057500,46.057500,0.0,0.007500,-0.132500,-0.187500,-0.130001,...,0.000000,46.153711,0.000000,0.000000,0.000000,0.000000,-0.389294,-0.390054,-0.389294,1
2019-05-14 04:00:00-04:00,46.565000,46.735000,46.565000,46.640000,46.640000,0.0,0.322500,0.330000,0.190000,0.135000,...,0.000000,46.373640,0.000000,0.000000,0.000000,0.000000,1.264723,1.256793,0.870506,0
2019-05-14 05:00:00-04:00,46.625000,46.750000,46.577500,46.750000,46.750000,0.0,0.060000,0.382500,0.390000,0.250000,...,0.000000,46.539028,0.000000,0.000000,0.000000,0.000000,0.235849,0.235571,1.108408,1
2019-05-14 06:00:00-04:00,46.725000,46.825000,46.725000,46.800000,46.800000,0.0,0.100000,0.160000,0.482500,0.490000,...,0.000000,46.653430,0.000000,0.000000,0.000000,0.000000,0.106952,0.106895,1.216545,1
2019-05-14 07:00:00-04:00,46.792500,47.075000,46.790000,46.992500,46.992500,0.0,0.067500,0.167500,0.227500,0.550000,...,0.056875,46.799702,0.000000,0.000000,0.000000,0.000000,0.411325,0.410481,1.632874,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-06 08:00:00-04:00,128.210000,128.310000,127.780000,127.940000,127.940000,0.0,-0.130000,-0.010000,-0.330000,-0.030000,...,-0.106235,128.626202,-1.113995,-50.481265,-30.729011,-19.752254,-0.202808,-0.203014,176.701811,1
2021-05-06 09:00:00-04:00,127.940000,129.799700,127.700000,127.900000,127.900000,0.0,-0.270000,-0.400000,-0.280000,-0.600000,...,-0.006407,128.610163,-0.779643,-54.747495,-35.532708,-19.214787,-0.031265,-0.031270,176.615301,1
2021-05-06 09:30:00-04:00,127.680000,128.550003,127.129997,127.830002,127.830002,19096274.0,-0.260000,-0.530000,-0.660000,-0.540000,...,-0.040732,128.599900,-0.195107,-3.731014,-29.172369,25.441355,-0.054729,-0.054744,176.463913,1


In [28]:
y = data['class_column']
features = [x for x in data.columns if x not in ['class_column']]
x = data[features]
scaler = MinMaxScaler()
# x = pd.DataFrame(scaler.fit_transform(x.values), columns=x.columns, index=x.index)
x_train= x.iloc[:-ROWS_TO_PREDICT]
y_train= y.iloc[:-ROWS_TO_PREDICT]
x_test =x.iloc[-ROWS_TO_PREDICT:]
y_test=y.iloc[-ROWS_TO_PREDICT:]

In [29]:
classifiers = dict()

classifiers['DecisionTreeClassifier 1'] = DecisionTreeClassifier(max_depth=10, random_state=0,criterion='gini',splitter='best')
classifiers['DecisionTreeClassifier 2'] = DecisionTreeClassifier(max_depth=10, random_state=0,criterion='gini',splitter='random')
classifiers['DecisionTreeClassifier 3'] = DecisionTreeClassifier(max_depth=10, random_state=0,criterion='entropy',splitter='best')
classifiers['RandomForestClassifier 4'] = RandomForestClassifier(n_estimators=1000, max_depth=3, random_state=0,criterion='gini', n_jobs = -1)
classifiers['RandomForestClassifier 5'] = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0,criterion='entropy', n_jobs = -1)
classifiers['GradientBoostingClassifier 1'] = GradientBoostingClassifier(n_estimators=100,random_state=0,criterion='friedman_mse',max_depth=3, learning_rate=0.1)
classifiers['GradientBoostingClassifier 2'] = GradientBoostingClassifier(n_estimators=1000,random_state=0,criterion='friedman_mse',max_depth=3, learning_rate=0.1)

In [30]:
def print_conf_matrix(test_y, predict, name):
    matrix = confusion_matrix(test_y, predict,  labels=[-1, 0, 1])
    print(matrix)
    ax= plt.subplot()
    sn.heatmap(matrix, annot=True, ax = ax) #annot=True to annotate cells

    # labels, title and ticks
    ax.set_xlabel('Predicted labels', color='white')
    ax.set_ylabel('True labels', color='white')
    ax.set_title(f'Confusion Matrix for {name}' , color='white')
    ax.xaxis.set_ticklabels(['-1','0', '1'], color='white')
    ax.yaxis.set_ticklabels(['-1','0', '1'], color='white')
    plt.show()

def train_model(model,train_x, train_y):
    model.fit(train_x, train_y)

In [31]:
predictions_train= dict()
score_train = dict()
predictions= dict()
score = dict()
for k,v in classifiers.items():
    print("Calculate: ", k)
    train_model(v,x_train,y_train)
    predictions_train[k] = v.predict(x_train)
    score_train[k] = accuracy_score(y_train.values, predictions_train[k])
    predictions[k] = v.predict(x_test)
    score[k] = accuracy_score(y_test.values, predictions[k])
    print('Score_train: ',  score_train[k] )
    print('Score: ',  score[k] )
    # print_conf_matrix(test_y, predictions[k], k)

headers = ["Classifier type", "Accuracy"]
score_df = pd.DataFrame(score.items(), columns=headers)
print(tabulate(score_df, headers, tablefmt="psql"))


Calculate:  DecisionTreeClassifier 1
Score_train:  0.6881905229560521
Score:  0.35294117647058826
Calculate:  DecisionTreeClassifier 2
Score_train:  0.6462067272280874
Score:  0.35294117647058826
Calculate:  DecisionTreeClassifier 3
Score_train:  0.6702676160078567
Score:  0.5
Calculate:  RandomForestClassifier 4
Score_train:  0.5094524920206236
Score:  0.47058823529411764
Calculate:  RandomForestClassifier 5
Score_train:  0.47557083231033637
Score:  0.38235294117647056
Calculate:  GradientBoostingClassifier 1
Score_train:  0.693346427694574
Score:  0.3235294117647059
Calculate:  GradientBoostingClassifier 2


KeyboardInterrupt: 

In [ ]:
filename_to_export = f'../data/results/{symbol}_1_{WINDOW}_{ROWS_TO_PREDICT}_{ datetime.now().strftime("%d_%m_%Y %H_%M_%S")}.csv'
score_df.to_csv(filename_to_export, index=False)

In [ ]:
rfe = RFE(classifiers['RandomForestClassifier 5'],10)
fited = rfe.fit(x_train, y_train)
rfe

In [ ]:
names = x.columns
columns=[]
for i in range(len(fited.support_)):
    if fited.support_[i]:
        columns.append(names[i])

print("Columns with predictive power:", columns )
columns = columns + ['high', 'low', 'volume', 'open']
columns

In [ ]:
x_test_cropped = x_test[columns]
x_train_cropped = x_train[columns]
x_train_cropped

In [ ]:
classifiers_boosted = dict()
classifiers_boosted['GradientBoostingClassifier 1'] = GradientBoostingClassifier(n_estimators=100,random_state=0,criterion='friedman_mse',max_depth=3, learning_rate=0.1)
classifiers_boosted['GradientBoostingClassifier 2'] = GradientBoostingClassifier(n_estimators=1000,random_state=0,criterion='friedman_mse',max_depth=3, learning_rate=0.3)
classifiers_boosted['GradientBoostingClassifier 3'] = GradientBoostingClassifier(n_estimators=1000,random_state=0,criterion='friedman_mse',max_depth=2, learning_rate=0.5)
classifiers_boosted['GradientBoostingClassifier 4'] = GradientBoostingClassifier(n_estimators=1000,random_state=0,criterion='friedman_mse',max_depth=2, learning_rate=0.8)

In [ ]:
predictions_train= dict()
score_train = dict()
predictions= dict()
score = dict()
for k,v in classifiers_boosted.items():
    print("Calculate: ", k)
    train_model(v,x_train_cropped,y_train)
    predictions_train[k] = v.predict(x_train_cropped)
    score_train[k] = accuracy_score(y_train.values, predictions_train[k])
    predictions[k] = v.predict(x_test_cropped)
    score[k] = accuracy_score(y_test.values, predictions[k])
    print('Score_train: ',  score_train[k] )
    print('Score: ',  score[k] )
    # print_conf_matrix(test_y, predictions[k], k)

headers = ["Classifier type", "Accuracy"]
score_df = pd.DataFrame(score.items(), columns=headers)
print(tabulate(score_df, headers, tablefmt="psql"))

In [ ]:
score_df.to_csv(filename_to_export,mode='a', index=False, header=False)